# Sheet 9

In [ ]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch.optim import Adam, SGD
from torch.utils.data import TensorDataset, DataLoader

## 1 Jet-Tagging with PyTorch
(a)

In [ ]:
# load the dijet data
features = np.load('data/dijet_features_normalized.npy').T
labels = np.load('data/dijet_labels.npy')

# shuffle the data
order = np.arange(len(labels))
np.random.seed(0)
np.random.shuffle(order)
features = features[order]
labels = labels[order]

n_features = features.shape[1]
print(n_features)
print(f'{features.shape=}, {labels.shape=}')

# TODO: create train, val and test splits

# TODO: create the datsets

(b)

In [ ]:
# TODO: Instantiate an MLP as specified

(c)

In [ ]:
# TODO: create the optimizer

# TODO: create the criterion

(d)

In [ ]:
# TODO: create the data loaders, use a batch size of 16

# TODO: Implement the training loop, validating after every epoch and keeping track of losses and accuracies

for epoch in range(30):
    for i, (x, y) in enumerate(train_loader):
        #TODO: train step
            
    # TODO: validate
            
# TODO: Plot the losses and accuracies.

(e)

In [ ]:
# TODO: load the best model and evaluate it on the test set


(f)

In [ ]:
# TODO: experiment with some hyperparameters and discuss your results


## Task 2: CNNs for Galaxy Classification

Read through and run the cells below to get started

In [ ]:
# download the data 
# it's ~200MB, might take some time depending on your internet speed

import urllib.request
_, msg = urllib.request.urlretrieve(
    "http://www.astro.utoronto.ca/~bovy/Galaxy10/Galaxy10.h5", 
    "data/Galaxy10.h5"
)

In [ ]:
import h5py
import numpy as np
import torch

label_names = [
    'Disk, Face-on, No Spiral',
    'Smooth, Completely round',
    'Smooth, in-between round',
    'Smooth, Cigar shaped',
    'Disk, Edge-on, Rounded Bulge',
    'Disk, Edge-on, Boxy Bulge',
    'Disk, Edge-on, No Bulge',
    'Disk, Face-on, Tight Spiral',
    'Disk, Face-on, Medium Spiral',
    'Disk, Face-on, Loose Spiral'
]
n_classes = len(label_names)

# To get the images and labels from file
with h5py.File('data/Galaxy10.h5', 'r') as F:
    images = np.array(F['images'])
    labels = np.array(F['ans'])
with h5py.File('data/Galaxy10_compressed.h5', 'w') as f:
    f.create_dataset('images', data=images.astype(np.float16), compression='gzip')
    f.create_dataset('ans', data=labels.astype(np.int32), compression='gzip')
    
images = images.astype(np.float32)

# comply to (batch, channel, height, width) convention of pytorch
images = np.moveaxis(images, -1, 1)  
# convert to torch
images = torch.from_numpy(images)
labels = torch.from_numpy(labels)

print(f'{images.shape=}, {labels.shape=}')

print(labels.shape, images.shape)
print(f'\nNumber of samples per class:')
for label, count in zip(*np.unique(labels, return_counts=True)):
    print(f'{label_names[label]:40s}: {count}')

In [ ]:
from matplotlib import pyplot as plt

# plot some samples of each class
itemindex = torch.where(labels == 3)
itemindex[:3]

samples_per_class = 3
fig, axss = plt.subplots(samples_per_class, n_classes, figsize=(n_classes * 2, samples_per_class * 2))
for label, (label_name, axs) in enumerate(zip(label_names, axss.T)):
    idx = torch.where(labels==label)[0][:samples_per_class] # take samples_per_class first occurences
    for i, (ind, ax) in enumerate(zip(idx, axs)):
        ax.imshow(images[ind].long().moveaxis(0, -1))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == 0:
            ax.set_title(label_name.replace(',',',\n'))
plt.tight_layout()
plt.show()

In [ ]:
from torchvision.transforms import Normalize

# Compute the mean and standard deviation per channel over the dataset
# stds = images.moveaxis(1, 0).reshape(3, -1).std(axis=1)
# means = images.moveaxis(1, 0).reshape(3, -1).mean(axis=1)

# Use precomputed means and stds
stds, means = torch.tensor([37.5412, 31.3756, 26.3283]), torch.tensor([27.7014, 23.8241, 18.1425])

# TODO: Normalize the images
normalize = Normalize(means, stds)
images_normalized = normalize(images)

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

n_samples = len(labels)

# split the data in training and validation sets, stratifying by the labels
train_idx, val_idx = train_test_split(np.arange(n_samples), test_size=0.1, stratify=labels)

# create pytorch datasets for training and validation
train_dataset = TensorDataset(images_normalized[train_idx].float(), labels[train_idx].long())
val_dataset = TensorDataset(images_normalized[val_idx].float(), labels[val_idx].long())

(a)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

#TODO: implement a small CNN as specified on the sheet


(b)

In [ ]:
# TODO: create DataLoaders for train and val, use a batch size of 16

# TODO: instantiate the model, optimizer and criterion

# TODO: implement the training loop, validating after every epoch, and make the requested plots


(c)

In [ ]:
# TODO: redo (c) with a ResNet
# Hint: Training is probably quicker on google colab (https://colab.research.google.com/)

